# VacationPy
----

#### Note
* Uses Google API - use https://developers.google.com/maps/gmp-get-started to acquire an api key and https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import gmaps_key

### Functions

In [2]:
# Return the most recent date from the the list provided
def getMostRecentDir(dir_list):
    
    dates = [int(date) for date in map(formatDate,dir_list)]
    most_rec = str(max(dates))
    return formatDate(most_rec)

In [3]:
# Flips the format of the date from MMDDYYYY to YYYYMMDD and vice versa
def formatDate(date):
    return date[4:] + date[:4]

In [4]:
# Compares the dataframe with the preference criteria
def compareDFtoPref(df,prefs):
    checklist = ["Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
    # check each column in checklist...
    for col in checklist:
        df = df.loc[df[col] >= prefs[col][0],:]
        df = df.loc[df[col] <= prefs[col][1],:]
    return df

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# Get most recent directory in output
output_path = os.path.join("..","WeatherPy","output")
output_dir_list = os.listdir(output_path)
recent_dir = getMostRecentDir(output_dir_list)

# Access the most recent directory and store the cities.csv file
weather_csv_path = os.path.join(output_path,recent_dir,"cities.csv")
weather_data = pd.read_csv(weather_csv_path)
weather_df = pd.DataFrame(weather_data)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hong Kong,22.29,114.16,64.99,75,14,7.29,HK,1609853050
1,Vakhrushev,48.98,142.97,-3.82,85,0,5.50,RU,1609853181
2,Khanbari,27.37,87.20,61.36,55,100,1.01,NP,1609853182
3,Chardara,41.25,67.97,29.53,77,28,5.64,KZ,1609853183
4,Port Alfred,-33.59,26.89,78.01,83,25,3.00,ZA,1609853184


### Humidity Heatmap

In [6]:
# Configure gmaps
gmaps.configure(api_key = gmaps_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"]

In [7]:
# Add Heatmap layer to map
fig = gmaps.figure(layout={"height":"600px","width":"900px"},center=(30.0, 0.0),zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=humidity.max(), point_radius=4)
fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# my personal preferences
preferences = {
    "Max Temp": [60,70],
    "Humidity": [10,40],
    "Cloudiness": [0,50],
    "Wind Speed": [0,10]
}

### Hotel Map

In [9]:
# Store into variable named `hotel_df`
hotel_df = compareDFtoPref(weather_df, preferences)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
295,Rawson,-43.30,-65.10,65.23,36,0,6.89,AR,1609853466
425,Buin,-33.73,-70.75,66.20,37,0,4.70,CL,1609853727
521,Rāpar,23.57,70.63,65.53,40,0,6.26,IN,1609853846
528,Chabahar,25.29,60.64,67.21,35,0,9.08,IR,1609853854


In [10]:
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

In [11]:
# Hit the Google Places API for each city's coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "key": gmaps_key,
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotels"
}

for i, row in hotel_df.iterrows():
    # store city's latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # call Google Places API using base url and parameters
    response = requests.get(base_url, params=params).json()

    # Store the first Hotel result into the DataFrame
    try:
        hotel_df.loc[i, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("No hotels within {params['radius']} meters found")

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
295,Rawson,-43.30,-65.10,65.23,36,0,6.89,AR,1609853466,Hotel Punta León
425,Buin,-33.73,-70.75,66.20,37,0,4.70,CL,1609853727,Paihuen
521,Rāpar,23.57,70.63,65.53,40,0,6.26,IN,1609853846,Shreemati Khetbai Ramji Momaya Gala Senotorium
528,Chabahar,25.29,60.64,67.21,35,0,9.08,IR,1609853854,Shahanhotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', width='900px'))